---
Vérifier qu'on n'est pas en 3.10.2 (sinon PyTorch ne marchera pas)

In [10]:
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


---
Imports - notemment de clubs_gym (https://github.com/fschlatt/clubs_gym)

In [11]:
import gym
import clubs_gym
import time
import random
import numpy as np
import pickle
from tqdm import tqdm

---
Liste des environnements disponibles

In [12]:
clubs_gym.ENVS

['LeducTwoPlayer-v0',
 'KuhnTwoPlayer-v0',
 'KuhnThreePlayer-v0',
 'LimitHoldemTwoPlayer-v0',
 'LimitHoldemSixPlayer-v0',
 'LimitHoldemNinePlayer-v0',
 'NoLimitHoldemTwoPlayer-v0',
 'NoLimitHoldemSixPlayer-v0',
 'NoLimitHoldemNinePlayer-v0',
 'NoLimitHoldemBbAnteNinePlayer-v0',
 'PotLimitOmahaTwoPlayer-v0',
 'PotLimitOmahaSixPlayer-v0',
 'PotLimitOmahaNinePlayer-v0',
 'ShortDeckTwoPlayer-v0',
 'ShortDeckSixPlayer-v0',
 'ShortDeckNinePlayer-v0']

Ne nous intéressera que `NoLimitHoldemTwoPlayer-v0` et `NoLimitHoldemSixPlayer-v0`

---
Ouvrons `NoLimitHoldemTwoPlayer-v0`.

In [13]:
env_str = "NoLimitHoldemTwoPlayer-v0"
num_players = 2  # heads-up
num_streets = 4  # texas hold'em
blinds = [1, 2]  # as on the tables on which we used to play @Jean
antes = 0  # a bit weird to have antes in a heads-up I reckon (single ints are expanded to the number of players so could've inputted a list)
raise_sizes = ['inf', 'inf', 'inf', 'inf']  # yes the documentation is wrong, 'inf' should be used and not float('inf')
num_raises = 'inf'  # max number of bets for each street* (including preflop)
num_suits = 4  # number of suits to use in deck (obviously 4 otherwise I mean it's weird)
num_ranks = 13  # same, obviously 13 (A, 2, 3, 4, 5, 6, 7, 8, 9, T, J, Q, K)
num_hole_cards = 2  # **
mandatory_num_hole_cards = 0  # you can use the fives on the board and it will end up being a draw if your enemy does it too
num_community_cards = [0, 3, 1, 1]  # preflop, flop, turn, river
start_stack = 100  # as on the tables on which we used to play @Jean (actually it was any integer between 80 and 200)
n_cards_for_hand = 5  # texas hold'em
order = ['sf', 'fk', 'fh', 'fl', 'st', 'tk', 'tp', 'pa', 'hc']  # most likely optional, but just to make sure we're working with the right order

config_dict = {'num_players': num_players,
               'num_streets': num_streets,
               'blinds': blinds,
               'antes': antes,
               'raise_sizes': raise_sizes,
               'num_raises': num_raises,
               'num_suits': num_suits,
               'num_ranks': num_ranks,
               'num_hole_cards': num_hole_cards,
               'mandatory_num_hole_cards': mandatory_num_hole_cards,
               'num_community_cards': num_community_cards,
               'start_stack': start_stack,
               'num_cards_for_hand': n_cards_for_hand,
               'order': order}


---
Un peu de vocabulaire pour les zguegs (là où y avait des * au-dessus):
- \* A card that’s dealt in a particular betting round, in many poker variants, is called a **street**. For example, in Texas Hold’em and Omaha, the community cards are referred to as the flop, turn and river, with the turn and river sometimes referred to as Fourth Street and Fifth Street.
- ** The term **hole cards' in poker signals that the cards are private cards that should only be viewed by the player. The dealer deals hole cards face down and players should ensure that the value of those cards remains secret.

---
`obs` est illisible donc on fait une fonction `print_obs` pour avoir des jolis prints sur ce qu'il se passe.

In [14]:
def print_obs(obs):
    """
    Fancy print of stats for the clubs gym poker gym.
    All Cards displayed (on voit les cartes de tout le monde quoi)
    
    param: obs: le premier chose renvoyé par env.step
    return: nada
    """
    for _ in range(5):
        print('---')
    print('------------------- Players\' stacks -------------------')
    print(f"player_0: {obs['stacks'][0]}\n" + \
          f"player_1: {obs['stacks'][1]}")
    print('------------------- Game Stats -------------------')
    print(f'community_cards: {obs["community_cards"]}')
    print(f'Dealer/Button: {obs["button"] % 2}')
    print(f'Pot: {obs["pot"]}')
    print(f'Commits: player_0: {obs["street_commits"][0]} |||  player_1: {obs["street_commits"][1]}')

    if not obs['action'] == -1:
        print(f"\n---------------- player {obs['action']}'s turn ----------------")
        print(f'hole_cards: {obs["hole_cards"]}')
        print(f'call: {obs["call"]}, min_raise: {obs["min_raise"]}, max_raise: {obs["max_raise"]}', end='\n\n')
    else:
        print('\n[INFO] - End of the Game.')

---
Main

Dans `env.step`, mettre un integer qui correspond à ce que vous voulez miser. 
- Si la mise est inférieure à la mise minimum, vous vous couchez. 
- Si la mise est supérieure à la mise maximum (en no limit, ça veut dire vous voulez miser plus que ce que vous détenez) alors all in.

Le truc ci-dessous vous fait changer de joueur pré/post flop - désolé. This is because *in Heads Up poker, the button acts first preflop and last postflop*.

In [15]:
from clubs_gym.agent.base import BaseAgent

class RandomAllInFold(BaseAgent):
    '''
    this agent bets everyting or folds (50/50 chance)
    '''
    def __init__(self, player_id, seed = 42):
        self.rand_generator = np.random.RandomState(seed)
        self.player_id = player_id

    def act(self, info_dict):
        available_chips = info_dict['stacks'][self.player_id]
        draw = np.random.random()
        if draw >= 0.5:
            action = 1 
        else:
            action = 0

        ## outpu  
        if action == 0:
            return(0)
        else:
            return(available_chips)

In [16]:
from clubs_gym.agent.base import BaseAgent
class HumanRandomAllInFold(BaseAgent):
    """
    only two possible actions, all in or folds
    """
    def __init__(self, player_id, seed = 42):
        self.player_id = player_id

    def act(self, info_dict):
        available_chips = info_dict['stacks'][self.player_id]
        action = -1
        while action != 0 and action != 1:
            action = int(input())
        if action == 0:
            return(0)
        else:
            return(available_chips)

In [17]:
class AllInFoldQLearningAgent(BaseAgent):
    """
    actions:
        - 0: folds
        - 1: all-in
    """
    def __init__(self, player_id, agent_init_info):
        self.player_id = player_id

        self.state_size = agent_init_info["state_size"]
        self.num_actions = agent_init_info["num_actions"]
        self.epsilon = agent_init_info["epsilon"]
        self.step_size = agent_init_info["step_size"]
        self.discount = agent_init_info["discount"]
        self.rand_generator = np.random.RandomState(agent_init_info["seed"])
        self.step = 0
        self.has_made_a_decision = False 
        self.q = {}

        
    def agent_start(self, state):
        self.has_made_a_decision = True

        tuple_representing_state = (str(state[0]), str(state[1]))
        current_q = self.q.setdefault(tuple_representing_state, [0]*self.num_actions)
        if self.rand_generator.rand() < self.epsilon:
            action = self.rand_generator.randint(self.num_actions) # random action selection
        else:
            action = self.argmax(current_q) # greedy action selection
        self.prev_state = tuple_representing_state
        self.prev_action = action
        return action
    
    def agent_step(self, reward, state):
        # Choose action using epsilon greedy.
        tuple_representing_state = (str(state[0]), str(state[1]))
        current_q = self.q.setdefault(tuple_representing_state, [0]*self.num_actions)
        if self.rand_generator.rand() < self.epsilon:
            action = self.rand_generator.randint(self.num_actions)
        else:
            action = self.argmax(current_q)
        
        previous_values_list = self.q[self.prev_state]
        previous_values_list[self.prev_action] += self.step_size*(reward + self.discount*np.max(self.q[tuple_representing_state]) - self.q[self.prev_state][self.prev_action])
        self.q[self.prev_state] = previous_values_list
        
        self.prev_state = tuple_representing_state
        self.prev_action = action
        return action
    
    def agent_end(self, reward):
        if self.has_made_a_decision:
            previous_values_list = self.q[self.prev_state]
            previous_values_list[self.prev_action] += self.step_size*(reward - self.q[self.prev_state][self.prev_action])
            self.q[self.prev_state] = previous_values_list



    def act(self, info_dict):
        reward = info_dict['rewards'][self.player_id]
        done = info_dict['done']
        state = info_dict['state']
        number_of_chips = info_dict['stacks'][self.player_id]
        

        if self.step == 0:
            action = self.agent_start(state)

        elif not done:
            action = self.agent_step(reward, state)

        elif done:
            print("weird")
            exit()

        self.step += 1
        if action == 0:
            return(0)
        else:
            return(number_of_chips)
        
    def argmax(self, q_values):
        top = float("-inf")
        ties = []

        for i in range(len(q_values)):
            if q_values[i] > top:
                top = q_values[i]
                ties = []

            if q_values[i] == top:
                ties.append(i)

        return self.rand_generator.choice(ties)

    def set_epsilon(self, value):
        self.epsilon = value

    @staticmethod
    def load(path):
        obj = pickle.load(open(path,'wb'))
        return(obj)

In [18]:
clubs_gym.envs.register({env_str: config_dict})
env = gym.make(env_str)
q_learning_agent_info = {"state_size":2, "num_actions": 2 , "epsilon": 0.1, "step_size": 0.1, "discount": 1.0, "seed": 0}
agents = [AllInFoldQLearningAgent(0, q_learning_agent_info), RandomAllInFold(1)]
env.register_agents(agents)

# Start the game
for episodes in tqdm(range(100000), position = 0):
    obs = env.reset(reset_stacks = True)

    # create a dict containing all necessary info for q learning agent
    info_dict = {}
    if obs['action'] != -1:
        done_agent = False
    else:
        done_agent = True
    info_dict['rewards'] = [0,0]
    info_dict['done'] = done_agent
    info_dict['state'] = obs['hole_cards']
    info_dict['stacks'] = obs['stacks']

    while True:
        bet = env.act(info_dict)
        obs, rewards, done, info = env.step(bet)  
        # updata dict
        info_dict = {}
        if obs['action'] != -1:
            done_agent = False
        else:
            done_agent = True
        info_dict['rewards'] = rewards
        info_dict['done'] = done_agent
        info_dict['state'] = obs['hole_cards']
        info_dict['stacks'] = obs['stacks']


        # print_obs(obs)
        if all(done):
            agents[0].agent_end(rewards[0])
            break

c:\Users\33631\Documents\Etudes\centrale 3A\RL\reinforcement-learning-poker\venv\lib\site-packages\gym\envs\registration.py:595: UserWarning: WARN: Overriding environment NoLimitHoldemTwoPlayer-v0
  logger.warn(f"Overriding environment {id}")
100%|██████████| 100000/100000 [00:16<00:00, 6110.86it/s]


In [19]:
print(agents[0].q['A♥', 'A♦'])
print(agents[0].q['2♥', '7♦'])

[2.9789140154906946, 33.914494116080384]
[0.008594183914281892, -9.0]
